## Initialize

In [ ]:

import requests
import pandas as pd
import os
import sys
from datetime import datetime
from ipywidgets import Dropdown
import json
import base64
import requests
from dateutil.parser import parse
import fitz


from PyPDF2 import PdfFileReader
from docx2pdf import convert
import docx
# import docs



sys.path.append(os.path.join(sys.path[0], '..', '..', '..', '..'))
import src.esh_client as esh
from demo.shared.tools import TENANT_SUFFIX, get_base_url, get_root_path

headers = {'Content-Type': 'application/json'}
base_url = get_base_url()

def get_doc_type(filename):
    print(filename)



## Choose file folder

In [ ]:
folder_path = os.path.join('..')
options = next(os.walk(folder_path))[1]
options.remove('notebooks')
selected = Dropdown(
    options=options,
    description='Select:',
    value = None,
    disabled=False,
)
selected

## Take all files from folder

In [ ]:
if selected.value:
    payload_list = []
    # Get files path
    current_path = os.path.join(sys.path[0], '..', selected.value)
    # Iterate about the folder with all files
    for filename in os.listdir(current_path):
        docType = ''
        filename_blank = ''
        created_at_string = ''
        changed_at_string = ''
        author = ''
        encoded_img = ''
        encoded_doc = ''
        # Read PDF File and encode to base64
        with open(os.path.join(current_path, filename), 'rb') as f:  # open in readonly mode
            if filename.endswith('pdf'):
                docType = 'PDF'
                filename_blank = filename.replace('.pdf', '')

                # encode to base64
                encoded_string = base64.b64encode(f.read())
                encoded_doc = encoded_string.decode('ascii')

                # Get file info
                pdf = PdfFileReader(f)
                info = pdf.getDocumentInfo()

                created_at = str(info.creation_date_raw)
                created_at_string = created_at[2:10]
                changed_at = str(info.modification_date_raw)
                changed_at_string = changed_at[2:10]
                print(filename_blank + ' Created at: ' + created_at_string +
                      ' Changed at: ' + changed_at_string)
                if info.author == None:
                    author = 'Not specified'
                else:
                    author = info.author

                # get image
                pdf_file = fitz.open(os.path.join(
                    current_path, filename))
                page = pdf_file[0]
                image_page = page.get_pixmap()
                image_page.shrink(2)
                encoded_string_img = base64.b64encode(image_page.tobytes())
                encoded_img = encoded_string_img.decode('ascii')
                encoded_img = 'data:;base64,' + encoded_img

                
            elif filename.endswith('doc'):
                
                docType = 'DOC'
                filename_blank = filename.replace('.doc', '')

                # encode to base64
                encoded_string = base64.b64encode(f.read())
                encoded_doc = encoded_string.decode('ascii')
                  
            elif filename.endswith('docx'):
                docType = 'DOCX'
                filename_blank = filename.replace('.docx', '')

                # encode to base64
                encoded_string = base64.b64encode(f.read())
                encoded_doc = encoded_string.decode('ascii')
                
                                
            elif filename.endswith('xls'):
                docType = 'XLS'
                filename_blank = filename.replace('.xls', '')

                # encode to base64
                encoded_string = base64.b64encode(f.read())
                encoded_doc = encoded_string.decode('ascii')
                
                                
            elif filename.endswith('xlsx'):
                docType = 'XLSX'
                filename_blank = filename.replace('.xlsx', '')

                # encode to base64
                encoded_string = base64.b64encode(f.read())
                encoded_doc = encoded_string.decode('ascii')
                
                                
            elif filename.endswith('msg'):
                docType = 'MSG'
                filename_blank = filename.replace('.msg', '')

                # encode to base64
                encoded_string = base64.b64encode(f.read())
                encoded_doc = encoded_string.decode('ascii')
                
                                
            # Append to JSON
            payload = json.dumps({
                'Document': [
                    {
                        'image': encoded_img,
                        'title': filename_blank,
                        'author': author,
                        'text': encoded_doc,
                        'createdAt': created_at_string,
                        'changedAt': changed_at_string,
                        'docType': docType
                    }
                ]
            }, indent=2
            )
            payload_list.append(payload)
            
else:
    print('No folder selected for files')


## Post data to server

In [ ]:
for element in payload_list:
    response = requests.request("POST", f'{base_url}/v1/data/DocResearchDemoExample', headers=headers, data=element)

    print(response.text)